In [1]:
import numpy as np

---
### Setting up basic matrices:
The Pauli matrices are the four 2x2 matrices, I, X, Y, Z.
For this example, we have a 4x4 Hamiltonian that is decomposed into IZ, ZI, XX, YY, and ZZ terms (this notation is not matrix multiplication--it is a Kronecker product, which takes two 2x2 matrices and gives a 4x4 matrix).

In [24]:
I = np.eye(2)
X = np.array([[0., 1.],
              [1., 0.]])
Y = np.array([[0., -1j],
              [1j, 0.]])
Z = np.array([[1., 0.],
              [0., -1.]])

In [25]:
IZ = np.kron(I, Z)
print(IZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0. -0.  0. -1.]]


In [26]:
ZI = np.kron(Z, I)
print(ZI)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1. -0.]
 [ 0.  0. -0. -1.]]


In [27]:
XX = np.kron(X, X)
print(XX)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [28]:
YY = np.kron(Y, Y)
print(YY)

[[ 0.+0.j  0.-0.j  0.-0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.-0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]]


In [29]:
ZZ = np.kron(Z, Z)
print(ZZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0. -1. -0.]
 [ 0. -0. -0.  1.]]


----
### Hamiltonian
We consider the Hamiltonian:
$H = IZ + ZI - XX - YY + ZZ$

In [30]:
H = IZ + ZI - XX - YY + ZZ
print(H)

[[ 3.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]]


---
### Measuring an observable
An observable is a matrix that we can measure against a state $\vec{\psi}$. The Hamiltonian is an observable. So are the Pauli terms that it is written as a sum of.

To measure some observable matrix $M$ with respect to a state $\vec{\psi}$, we just compute $\vec{\psi}^{\dagger} M \vec{\psi}$. The $\dagger$ means conjugate transpose = transpose the matrix and then take complex conjugates. In this specific example, all the numbers are real, so it happens to be the same as an ordinary matrix transpose.

Here is a helper method that does everything for us:

In [43]:
def expected_value(M, state):
    return np.asmatrix(state).getH() @ M @ state  # .getH() does conjugate tranpose

---
### Let's try for $\vec{\psi} = [0, 1, 0, 0]^T$

In [36]:
state = np.array([[0.], [1.], [0.], [0.]])

#### Method 1: measure $H$ directly
A quantum computer can't actually measure $H$ directly, but let's just check what the answer should be.

In [39]:
print(expected_value(H, state))

[[-1.+0.j]]


#### Method 2: linearity of expectation
A quantum computer can compute the Pauli observables though. So let's use linearity of expectation instead:

In [42]:
IZ_expectation = expected_value(IZ, state)
ZI_expectation = expected_value(ZI, state)
XX_expectation = expected_value(XX, state)
YY_expectation = expected_value(YY, state)
ZZ_expectation = expected_value(ZZ, state)
print(IZ_expectation, ZI_expectation, XX_expectation, YY_expectation, ZZ_expectation)
print(IZ_expectation + ZI_expectation - XX_expectation - YY_expectation + ZZ_expectation)

[[-1.]] [[1.]] [[0.]] [[0.+0.j]] [[-1.]]
[[-1.+0.j]]


Yup, we got -1 both ways!

---
### Variances / Covariances

In [150]:
def variance(M, state):
    # variances is <M^2> - <M>^2
    return expected_value(M @ M, state) - expected_value(M, state) ** 2

def covariance(M_1, M_2, state):
    return expected_value(M_1 @ M_2, state) - expected_value(M_1, state) * expected_value(M_2, state)

The cells below show that:
- Var(H) = 4
- Var(IZ) + Var(ZI) + Var(XX) + Var(YY) + Var(ZZ) = 2 -- not equal because we need to account for covariances
- the summed covariances do indeed equal 4. Note that the XX and ZZ

In [65]:
variance(H, state)

matrix([[4.+0.j]])

In [66]:
variance(IZ, state)

matrix([[0.]])

In [67]:
variance(ZI, state)

matrix([[0.]])

In [68]:
variance(XX, state)

matrix([[1.]])

In [69]:
variance(YY, state)

matrix([[1.+0.j]])

In [70]:
variance(ZZ, state)

matrix([[0.]])

In [98]:
terms, coefficients = [IZ, ZI, XX, YY, ZZ], [1., 1., -1., -1., 1.]
total = np.array([[0.+0.j]])
for i in range(5):
    for j in range(5):
        cov = covariance(terms[i], terms[j], state)
        if cov != 0:
            print(cov, i, j)
            total += (coefficients[i] * coefficients[j] * covariance(terms[i], terms[j], state))
print('Variance from summed covariances is %s' % total)

[[1.]] 2 2
[[1.+0.j]] 2 3
[[1.+0.j]] 3 2
[[1.+0.j]] 3 3
Variance from summed covariances is [[4.+0.j]]


----
### Different State, $\vec{\psi} = [0.43 - 0.16i, -0.49, 0.44i, -0.39 + 0.46i]$

In [161]:
state = np.array([[0.43 - 0.16j], [-0.49], [0.43j], [-0.39 + 0.46j]])

In [162]:
variance(H, state)

matrix([[4.35848816-2.22044605e-16j]])

In [166]:
variance(IZ, state), variance(ZI, state), variance(-XX, state), variance(-YY, state), variance(ZZ, state)

(matrix([[0.95576944+0.j]]),
 matrix([[0.989596+0.j]]),
 matrix([[0.76629724+0.j]]),
 matrix([[0.76629724+0.j]]),
 matrix([[0.97693936+0.j]]))

In [167]:
print(covariance(IZ, ZI, state))
print(covariance(IZ, ZZ, state))
print(covariance(ZI, ZZ, state))
print(covariance(-XX, -YY, state))
print(covariance(-XX, ZZ, state))
print(covariance(-YY, ZZ, state))

[[0.1287768+0.j]]
[[-0.06690672+0.j]]
[[-0.1937784+0.j]]
[[0.08370276+0.j]]
[[0.41059608+0.j]]
[[-0.41059608+0.j]]


In [165]:
# Setting 1 (measure all terms separately) would require a number of state preparations equal to:
5 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936)

22.2744964

In [170]:
# Setting 2 (measure {XX}, {YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
3 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.1287768)

14.6395752

In [172]:
# Setting 3 (measure {XX, YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
2 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.41059608 + 2*-0.41059608 + 2*0.1287768)

9.7597168